# Homework 1 - Invasive species competition
Entering another image processing competition for homework 1
https://www.kaggle.com/c/invasive-species-monitoring/leaderboard

In [1]:
%matplotlib inline

Define path to data: (It's a good idea to put it in a subdirectory of your notebooks folder, and then exclude that directory from git control by adding it to .gitignore.)

In [2]:
#path = "data/dogscats/"
#path = "data/dogscats/sample/"
path = "data/kaggle/"

A few basic libraries that we'll need for the initial exercises:

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

We have created a file most imaginatively called 'utils.py' to store any little convenience functions we'll want to use. We will discuss these as we use them.

In [4]:
import utils; reload(utils)
from utils import plots

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN not available)
Using Theano backend.


In [19]:
# sort out training set into healthy and invasive folders
import pandas as pd
import shutil as sh

df = pd.read_csv("data/kaggle/train_labels.csv")
print(df.head())
for i, row in df.iterrows():
    #print(row['name'],row.invasive)
    if row.invasive == 1:
        src = "data/kaggle/train/"+str(row['name'])+".jpg"
        dst = "data/kaggle/train/invasive/"
    else:
        src = "data/kaggle/train/"+str(row['name'])+".jpg"
        dst = "data/kaggle/train/healthy/"
    print(src,dst)
    try:
        sh.move(src,dst)
    except Exception as ex:
        print(ex)

   name  invasive
0     1         0
1     2         0
2     3         1
3     4         0
4     5         1
data/kaggle/train/1.jpg data/kaggle/train/healthy/
Destination path 'data/kaggle/train/healthy/1.jpg' already exists
data/kaggle/train/2.jpg data/kaggle/train/healthy/
Destination path 'data/kaggle/train/healthy/2.jpg' already exists
data/kaggle/train/3.jpg data/kaggle/train/invasive/
Destination path 'data/kaggle/train/invasive/3.jpg' already exists
data/kaggle/train/4.jpg data/kaggle/train/healthy/
Destination path 'data/kaggle/train/healthy/4.jpg' already exists
data/kaggle/train/5.jpg data/kaggle/train/invasive/
Destination path 'data/kaggle/train/invasive/5.jpg' already exists
data/kaggle/train/6.jpg data/kaggle/train/healthy/
data/kaggle/train/7.jpg data/kaggle/train/invasive/
data/kaggle/train/8.jpg data/kaggle/train/invasive/
data/kaggle/train/9.jpg data/kaggle/train/healthy/
data/kaggle/train/10.jpg data/kaggle/train/healthy/
data/kaggle/train/11.jpg data/kaggle/train/in

# Use a pretrained VGG model with our **Vgg16** class

Our first step is simply to use a model that has been fully created for us, which can recognise a wide variety (1,000 categories) of images. We will use 'VGG', which won the 2014 Imagenet competition, and is a very simple model to create and understand. The VGG Imagenet team created both a larger, slower, slightly more accurate model (*VGG  19*) and a smaller, faster model (*VGG 16*). We will be using VGG 16 since the much slower performance of VGG19 is generally not worth the very minor improvement in accuracy.

We have created a python class, *Vgg16*, which makes using the VGG 16 model very straightforward. 

## The punchline: state of the art custom model in 7 lines of code

Here's everything you need to do to get >97% accuracy on the Dogs vs Cats dataset - we won't analyze how it works behind the scenes yet, since at this stage we're just going to focus on the minimum necessary to actually do useful work.

In [20]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

In [21]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [23]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 1888 images belonging to 2 classes.
Found 407 images belonging to 2 classes.
Epoch 1/1
1888/1888 [==============================] - 380s - loss: 0.6192 - acc: 0.7728 - val_loss: 0.3313 - val_acc: 0.8600

## Predictions
Predict categories and save results for submission

In [26]:
# load test batch, make sure you put the test batch in a class subfolder
batches, preds = vgg.test(path+"test", batch_size = batch_size*2)

Found 1531 images belonging to 1 classes.


In [28]:
import csv

# get filenames
filenames = batches.filenames

with open("data/kaggle/my_submission.csv", "wb") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(('name','invasive'))
    for i, pred in enumerate(preds):
        id = filenames[i].replace('unknown/','').replace('.jpg','') 
        #label = np.argmax(pred)
        label = pred[1]
        print(id,label,pred)                
        csvwriter.writerow((id,label))

20 0.0235352 [ 0.9765  0.0235]
330 0.971767 [ 0.0282  0.9718]
162 0.0194072 [ 0.9806  0.0194]
871 0.397268 [ 0.6027  0.3973]
1320 0.362549 [ 0.6375  0.3625]
267 0.997144 [ 0.0029  0.9971]
1175 0.0503676 [ 0.9496  0.0504]
110 0.197535 [ 0.8025  0.1975]
664 0.99996 [  3.9629e-05   9.9996e-01]
1531 0.935682 [ 0.0643  0.9357]
737 0.0148846 [ 0.9851  0.0149]
1090 0.831215 [ 0.1688  0.8312]
118 0.0430409 [ 0.957  0.043]
164 0.0204361 [ 0.9796  0.0204]
114 0.199949 [ 0.8001  0.1999]
546 0.981017 [ 0.019  0.981]
881 0.00164286 [ 0.9984  0.0016]
1342 0.993973 [ 0.006  0.994]
310 0.695105 [ 0.3049  0.6951]
210 0.999999 [  7.9091e-07   1.0000e+00]
255 0.0199888 [ 0.98  0.02]
445 0.125379 [ 0.8746  0.1254]
1300 0.0214476 [ 0.9786  0.0214]
1052 0.00989561 [ 0.9901  0.0099]
1044 0.79692 [ 0.2031  0.7969]
161 0.317941 [ 0.6821  0.3179]
551 0.00327968 [ 0.9967  0.0033]
755 0.000868414 [  9.9913e-01   8.6841e-04]
1325 0.0192341 [ 0.9808  0.0192]
233 0.162645 [ 0.8374  0.1626]
422 0.0397247 [ 0.9603  0.